In [0]:
# Imports
from IPython.display import Image
import pandas as pd
import numpy as np
from scipy.stats import binom_test

In [0]:
PATH = '/content/drive/My Drive/Master/UFMG/Dissertation/Exploration/'

In [3]:
# READ CSV - Method 2 (Drive)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Read tables

### Patients table

In [0]:
mimic_patients_df = pd.read_csv(PATH + 'mimic-III/PATIENTS.csv')

### Admission table

In [0]:
mimic_admission_df = pd.read_csv(PATH + 'mimic-III/ADMISSIONS.csv')

#### Merge both tables and calculate true age

In [0]:
# Merge patient and admission tables
mimic_patients_admission_df = pd.merge(mimic_admission_df, mimic_patients_df, on = 'SUBJECT_ID' , how = 'inner')

In [0]:
# ADMITTIME and DOB objects to datetime
mimic_patients_admission_df["ADMITTIME"] = pd.to_datetime(mimic_patients_admission_df["ADMITTIME"], format='%Y%m%d %H:%M:%S')
mimic_patients_admission_df["DOB"] = pd.to_datetime(mimic_patients_admission_df["DOB"], format='%Y%m%d %H:%M:%S')

# True age
mimic_patients_admission_df["AGES"] = mimic_patients_admission_df["ADMITTIME"].sub(mimic_patients_admission_df["DOB"]).dt.days/365.242

# Adult patients only
mimic_patients_admission_df = mimic_patients_admission_df[(mimic_patients_admission_df.AGES >= 18)]

### ICD-9 Codes table

In [0]:
mimic_diagnoses_df = pd.read_csv(PATH + 'mimic-III/DIAGNOSES_ICD.csv')

In [0]:
mimic_patients_diagnoses_df = pd.merge(mimic_patients_admission_df, mimic_diagnoses_df, on = ['SUBJECT_ID', 'HADM_ID'], how = 'inner')

### ICD-9 Descriptions table

In [0]:
mimic_diagnoses_descriptions_df = pd.read_csv(PATH + 'mimic-III/D_ICD_DIAGNOSES.csv')

## Occurrence and Mortality

### By insurance

In [0]:
insurance_mortality_df = mimic_patients_diagnoses_df.groupby(['ICD9_CODE', 'INSURANCE', 'HOSPITAL_EXPIRE_FLAG']).size().unstack()
insurance_mortality_df = insurance_mortality_df.reset_index()
insurance_mortality_df.columns.names = [None]
insurance_mortality_df.columns = ['ICD9_CODE', 'INSURANCE', 'ALIVE', 'DEAD']
insurance_mortality_df.insert(4, 'TOTAL', 'NULL')
insurance_mortality_df = insurance_mortality_df.fillna(0)

# Compute alive, dead and total
for index, row in insurance_mortality_df.iterrows():
  insurance_mortality_df.at[index, 'TOTAL'] = row.ALIVE + row.DEAD
  
insurance_mortality_group = insurance_mortality_df.groupby('INSURANCE')['TOTAL'].sum()

In [12]:
print(insurance_mortality_group.sum())
print(insurance_mortality_group)

569086.0
INSURANCE
Government     13661.0
Medicaid       53044.0
Medicare      335271.0
Private       162557.0
Self Pay        4553.0
Name: TOTAL, dtype: float64


In [0]:
# 1. Mortality significance: only +30 patients dead
insurance_mortality_significance_df = insurance_mortality_df.loc[insurance_mortality_df.DEAD > 30].copy()

# 2. Remove ICD9 codes with only ONE insurance
for index, row in insurance_mortality_significance_df.iterrows():
  rows = insurance_mortality_significance_df.loc[insurance_mortality_significance_df.ICD9_CODE == row.ICD9_CODE]
  if (len(rows) == 1):
    insurance_mortality_significance_df.drop(rows.index, inplace=True)

#### Binomial test

In [0]:
# TODO:
# - REFACTORING!!!!! 

# 3. Binomial test
icd9_code, insurance, p_value_occ, p_value_mort = ([] for i in range(4))
medicare_t, medicaid_t, priv_t, gov_t, self_t = ([] for i in range(5))
medicare_d, medicaid_d, priv_d, gov_d, self_d = ([] for i in range(5))
icd_pass = True
 
for index, row in insurance_mortality_significance_df.iterrows():
  if row.ICD9_CODE not in icd9_code: # icd must be evaluate only once
    icd_pass = True

  if icd_pass:
    rows = insurance_mortality_significance_df[insurance_mortality_significance_df["ICD9_CODE"] == row.ICD9_CODE]
    icd_pass = False
    
    medicare_total, medicaid_total, priv_total, gov_total, self_total = 0, 0, 0, 0, 0
    medicare_dead, medicaid_dead, priv_dead, gov_dead, self_dead = 0, 0, 0, 0, 0
    medicare_percentange_dead, medicaid_percentange_dead, priv_percentange_dead, gov_percentange_dead, self_percentange_dead = 0, 0, 0, 0, 0
    
    if rows.loc[rows.INSURANCE == "Medicare"]["TOTAL"].any():
      medicare_total = rows.loc[rows.INSURANCE == "Medicare"]["TOTAL"].item()
      medicare_dead = rows.loc[rows.INSURANCE == "Medicare"]["DEAD"].item()
      medicare_percentange_dead = medicare_dead / medicare_total
      
    if rows.loc[rows.INSURANCE == "Medicaid"]["TOTAL"].any():
      medicaid_total = rows.loc[rows.INSURANCE == "Medicaid"]["TOTAL"].item()
      medicaid_dead = rows.loc[rows.INSURANCE == "Medicaid"]["DEAD"].item()
      medicaid_percentange_dead = medicaid_dead / medicaid_total
      
    if rows.loc[rows.INSURANCE == "Private"]["TOTAL"].any():
      priv_total = rows.loc[rows.INSURANCE == "Private"]["TOTAL"].item()
      priv_dead = rows.loc[rows.INSURANCE == "Private"]["DEAD"].item()
      priv_percentange_dead = priv_dead / priv_total
      
    if rows.loc[rows.INSURANCE == "Government"]["TOTAL"].any():
      gov_total = rows.loc[rows.INSURANCE == "Government"]["TOTAL"].item()
      gov_dead = rows.loc[rows.INSURANCE == "Government"]["DEAD"].item()
      gov_percentange_dead = gov_dead / gov_total

    if rows.loc[rows.INSURANCE == "Self Pay"]["TOTAL"].any():
      self_total = rows.loc[rows.INSURANCE == "Self Pay"]["TOTAL"].item()
      self_dead = rows.loc[rows.INSURANCE == "Self Pay"]["DEAD"].item()
      self_percentange_dead = gov_dead / gov_total
      
    medicare_percentage_pop = medicare_total/insurance_mortality_group.Medicare
    medicaid_percentage_pop = medicaid_total/insurance_mortality_group.Medicaid
    priv_percentage_pop = priv_total/insurance_mortality_group.Private
    gov_percentage_pop = gov_total/insurance_mortality_group.Government
    self_percentage_pop = self_total/insurance_mortality_group['Self Pay']
    
    # If less common in one insurance (considering total population) but they die more
    if ((medicare_total != 0 and medicare_dead != 0 and (medicare_percentage_pop < (medicaid_percentage_pop + priv_percentage_pop + gov_percentage_pop + self_percentage_pop)) and (medicare_percentange_dead > (medicaid_percentange_dead + priv_percentange_dead + gov_percentange_dead + self_percentange_dead))) or 
        (medicaid_total != 0 and medicaid_dead != 0 and (medicaid_percentage_pop < (medicare_percentage_pop + priv_percentage_pop + gov_percentage_pop + self_percentage_pop)) and (medicaid_percentange_dead > (medicare_percentange_dead + priv_percentange_dead + gov_percentange_dead + self_percentange_dead))) or
        (priv_total != 0 and priv_dead != 0 and (priv_percentage_pop < (medicare_percentage_pop + medicaid_percentage_pop + gov_percentage_pop + self_percentage_pop)) and (priv_percentange_dead > (medicare_percentange_dead + medicaid_percentange_dead + gov_percentange_dead + self_percentange_dead))) or
        (gov_total != 0 and gov_dead != 0 and (gov_percentage_pop < (medicare_percentage_pop + medicaid_percentage_pop + priv_percentage_pop + self_percentage_pop)) and (gov_percentange_dead > (medicare_percentange_dead + medicaid_percentange_dead + priv_percentange_dead + self_percentange_dead))) or
        (self_total != 0 and self_dead != 0 and (self_percentage_pop < (medicare_percentage_pop + medicaid_percentage_pop + priv_percentage_pop)) and (self_percentange_dead > (medicare_percentange_dead + medicaid_percentange_dead + priv_percentange_dead + gov_percentange_dead)))):
      
      # Values by insurance (for comparison)
      medicare_t.append(medicare_total), medicaid_t.append(medicaid_total), priv_t.append(priv_total), gov_t.append(gov_total), self_t.append(self_total)
      medicare_d.append(medicare_dead), medicaid_d.append(medicaid_dead), priv_d.append(priv_dead), gov_d.append(gov_dead), self_d.append(self_dead)

      # Occurrences
      n_occ = medicare_total + medicaid_total + priv_total + gov_total + self_total
      
      if ((medicare_percentage_pop < (medicaid_percentage_pop + priv_percentage_pop + gov_percentage_pop + self_percentage_pop)) and
          (medicare_percentange_dead > (medicaid_percentange_dead + priv_percentange_dead + gov_percentange_dead + self_percentange_dead))):
        # medicare
        x_occ = medicare_total
        x_mort = medicare_dead
        prob_occ = (n_occ - medicare_total)/n_occ
        prob_mort = (medicaid_percentange_dead + priv_percentange_dead + gov_percentange_dead + self_percentange_dead)/(n_occ - medicare_total)
        insurance.append("MEDICARE")
        icd9_code.append(rows.iloc[0]['ICD9_CODE'])
      elif ((medicaid_percentage_pop < (medicare_percentage_pop + priv_percentage_pop + gov_percentage_pop + self_percentage_pop)) and
            (medicaid_percentange_dead > (medicare_percentange_dead + priv_percentange_dead + gov_percentange_dead + self_percentange_dead))):
        # medicaid
        x_occ = medicaid_total
        x_mort = medicaid_dead
        prob_occ = (n_occ - medicaid_total)/n_occ
        prob_mort = (medicare_percentange_dead + priv_percentange_dead + gov_percentange_dead + self_percentange_dead)/(n_occ - medicaid_total)
        insurance.append("MEDICAID")
        icd9_code.append(rows.iloc[0]['ICD9_CODE'])
      elif ((priv_percentage_pop < (medicare_percentage_pop + medicaid_percentage_pop + gov_percentage_pop + self_percentage_pop)) and 
            (priv_percentange_dead > (medicare_percentange_dead + medicaid_percentange_dead + gov_percentange_dead + self_percentange_dead))):
        # private
        x_occ = priv_total
        x_mort = priv_dead
        prob_occ = (n_occ - priv_total)/n_occ
        prob_mort = (medicare_percentange_dead + medicaid_percentange_dead + gov_percentange_dead + self_percentange_dead)/(n_occ - priv_total)
        insurance.append("PRIVATE")
        icd9_code.append(rows.iloc[0]['ICD9_CODE'])      
      elif ((gov_percentage_pop < (medicare_percentage_pop + medicaid_percentage_pop + priv_percentage_pop + self_percentage_pop)) and 
            (gov_percentange_dead > (medicare_percentange_dead + medicaid_percentange_dead + priv_percentange_dead + self_percentange_dead))):
        # government
        x_occ = gov_total
        x_mort = gov_dead
        prob_occ = (n_occ - gov_total)/n_occ
        prob_mort = (medicare_percentange_dead + medicaid_percentange_dead + priv_percentange_dead + self_percentange_dead)/(n_occ - gov_total)
        insurance.append("GOVERNMENT")
        icd9_code.append(rows.iloc[0]['ICD9_CODE'])           
      else:
        # self pay
        x_occ = self_total
        x_mort = self_dead
        prob_occ = (n_occ - self_total)/n_occ
        prob_mort = (medicare_percentange_dead + medicaid_percentange_dead + priv_percentange_dead + gov_percentange_dead)/(n_occ - self_total)
        insurance.append("SELFPAY")
        icd9_code.append(rows.iloc[0]['ICD9_CODE'])
         
      # p-value: binom test
      p_value_occ.append(binom_test(x = x_occ, n = n_occ, p = prob_occ, alternative = 'two-sided'))
      p_value_mort.append(binom_test(x = x_mort, n = x_occ, p = prob_mort, alternative = 'two-sided'))

In [15]:
dict = {'Insurance': insurance, 'ICD9': icd9_code, 'Occ p-value': p_value_occ, 
        'Mort p-value': p_value_mort, 'Medicare Total': medicare_t, 'Medicare Dead': medicare_d, 
        'Medicaid Total': medicaid_t, 'Medicaid Dead': medicaid_d, 'Priv Total': priv_t,
        'Priv Dead': priv_d, 'Gov Total': gov_t, 'Gov Dead': gov_d, 'Self Total': self_t, 
        'Self Dead': self_d}  
    
binom_test_df = pd.DataFrame(dict)
binom_test_df = binom_test_df.merge(mimic_diagnoses_descriptions_df, left_on='ICD9', right_on='ICD9_CODE')
binom_test_df = binom_test_df.drop(['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE'], axis=1)

pd.set_option("display.max_colwidth", 100)

# null hypothesis rejected: p-value < 0.05
binom_test_df[(binom_test_df['Occ p-value'] < 0.05) & (binom_test_df['Mort p-value'] < 0.05)]

,Insurance,ICD9,Occ p-value,Mort p-value,Medicare Total,Medicare Dead,Medicaid Total,Medicaid Dead,Priv Total,Priv Dead,Gov Total,Gov Dead,Self Total,Self Dead,LONG_TITLE
0,MEDICARE,07054,3.608793e-24,5.719229e-129,442.0,60.0,0,0,304.0,38.0,0,0,0,0,Chronic hepatitis C without mention of hepatic coma
1,MEDICARE,1120,1.198658e-21,7.804211e-97,317.0,49.0,0,0,208.0,31.0,0,0,0,0,Candidiasis of mouth
3,MEDICARE,1977,1.192364e-04,1.297667e-280,373.0,124.0,0,0,322.0,87.0,0,0,0,0,"Malignant neoplasm of liver, secondary"
5,MEDICARE,1985,5.470397e-03,1.109543e-230,385.0,103.0,0,0,347.0,83.0,0,0,0,0,Secondary malignant neoplasm of bone and bone marrow
8,MEDICARE,2639,1.310863e-108,1.180970e-238,678.0,111.0,0,0,333.0,54.0,0,0,0,0,Unspecified protein-calorie malnutrition
9,MEDICARE,2720,0.000000e+00,0.000000e+00,3431.0,291.0,0,0,1867.0,74.0,0,0,0,0,Pure hypercholesterolemia
10,PRIVATE,2763,1.298047e-95,1.162241e-98,534.0,82.0,0,0,250.0,41.0,0,0,0,0,Alkalosis
11,MEDICARE,27652,3.090211e-118,6.790613e-214,739.0,96.0,0,0,363.0,42.0,0,0,0,0,Hypovolemia
12,MEDICARE,2768,1.251994e-62,1.887755e-187,682.0,80.0,0,0,409.0,37.0,0,0,0,0,Hypopotassemia
13,MEDICARE,2851,1.389990e-270,0.000000e+00,2396.0,225.0,0,0,1337.0,105.0,0,0,0,0,Acute posthemorrhagic anemia


In [16]:
print(len(binom_test_df[(binom_test_df['Occ p-value'] < 0.05) & (binom_test_df['Mort p-value'] < 0.05)]))

39


In [17]:
# Only cases with transplanted organs
pd.set_option("display.max_colwidth", 500)
binom_test_df.loc[binom_test_df.iloc[:, -1].str.contains(r'transplant')]

,Insurance,ICD9,Occ p-value,Mort p-value,Medicare Total,Medicare Dead,Medicaid Total,Medicaid Dead,Priv Total,Priv Dead,Gov Total,Gov Dead,Self Total,Self Dead,LONG_TITLE


In [18]:
# Only cases with heart problems - CODE ?
binom_test_df.loc[binom_test_df.iloc[:, -1].str.contains(r'heart')]

,Insurance,ICD9,Occ p-value,Mort p-value,Medicare Total,Medicare Dead,Medicaid Total,Medicaid Dead,Priv Total,Priv Dead,Gov Total,Gov Dead,Self Total,Self Dead,LONG_TITLE
